## Ami-Br

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from huggingface_hub import login
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from timm.layers import SwiGLUPacked
import pickle

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hugging Face login
login(token="your_huggingface_token_here")  # Replace with your Hugging Face token

# Load Virchow2 model
model_name = "hf-hub:paige-ai/Virchow2"
virchow_model = timm.create_model(model_name, pretrained=True, mlp_layer=SwiGLUPacked, act_layer=torch.nn.SiLU)
virchow_model.eval().to(device)
virchow_config = resolve_data_config(virchow_model.pretrained_cfg, model=virchow_model)
virchow_transform = create_transform(**virchow_config)

# Embedding extractor
def extract_embedding(img_path):
    image = Image.open(img_path).convert("RGB")
    image_tensor = virchow_transform(image).unsqueeze(0).to(device)
    with torch.inference_mode(), torch.autocast(device_type="cuda", dtype=torch.float16):
        output = virchow_model(image_tensor)
        class_token = output[:, 0]
        patch_tokens = output[:, 5:]
        embedding = torch.cat([class_token, patch_tokens.mean(1)], dim=-1).squeeze(0).to(torch.float32)
    return embedding.cpu()

# Inference dataset
class InferenceDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.embeddings = [extract_embedding(p) for p in tqdm(image_paths, desc="Extracting embeddings")]
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Classifier head
class VirchowBinaryClassifier(nn.Module):
    def __init__(self):
        super(VirchowBinaryClassifier, self).__init__()
        self.classifier = nn.Linear(2560, 1)

    def forward(self, x):
        return self.classifier(x)

# Load test data
test_root = "/data/MELBA-AmiBr/Datasets_Stratified/AMi-Br/Test"
class_map = {"Atypical": 0, "Normal": 1}
image_paths, labels = [], []

for class_name, label_val in class_map.items():
    class_folder = os.path.join(test_root, class_name)
    for fname in os.listdir(class_folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.tif')):
            image_paths.append(os.path.join(class_folder, fname))
            labels.append(label_val)

# Prepare dataset and dataloader
test_dataset = InferenceDataset(image_paths, labels)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8, pin_memory=True)

# Load trained fold models
num_folds = 5
model_paths = [f"virchow2_linear_probe_fold_{i + 1}_best.pth" for i in range(num_folds)]
models = []

for path in model_paths:
    model = VirchowBinaryClassifier().to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    model.eval()
    models.append(model)

# Inference and evaluation
true_labels = np.array(test_dataset.labels)
fold_bal_accs, fold_aurocs = [], []
fold_probs_dict = {}

for i, model in enumerate(models):
    fold_probs = []

    with torch.no_grad():
        for embeddings, _ in tqdm(test_loader, desc=f"Inference Fold {i + 1}"):
            embeddings = embeddings.to(device)
            logits = model(embeddings)
            probs = torch.sigmoid(logits).squeeze(1).cpu().numpy()
            fold_probs.extend(probs)

    fold_probs = np.array(fold_probs)
    fold_preds = (fold_probs > 0.5).astype(int)

    bal_acc = balanced_accuracy_score(true_labels, fold_preds)
    auroc = roc_auc_score(true_labels, fold_probs)

    fold_bal_accs.append(bal_acc)
    fold_aurocs.append(auroc)

    print(f"\nFold {i + 1} - Balanced Accuracy: {bal_acc:.4f}, AUROC: {auroc:.4f}")

    fold_probs_dict[f"fold_{i + 1}"] = {
        "probs": fold_probs,
        "preds": fold_preds,
        "true_labels": true_labels
    }

# Summary
mean_bal_acc = np.mean(fold_bal_accs)
std_bal_acc = np.std(fold_bal_accs)
mean_auroc = np.mean(fold_aurocs)
std_auroc = np.std(fold_aurocs)

print("\n--- Per-Fold Evaluation Summary (Virchow2) ---")
print(f"Balanced Accuracy: {mean_bal_acc:.4f} ± {std_bal_acc:.4f}")
print(f"AUROC: {mean_auroc:.4f} ± {std_auroc:.4f}")

# Save predictions
output_path = "virchow2_amibr_test_predictions.pkl"
with open(output_path, "wb") as f:
    pickle.dump(fold_probs_dict, f)

print(f"\nSaved fold predictions and labels to: {output_path}")


Extracting embeddings: 100%|██████████| 826/826 [00:18<00:00, 44.80it/s]
/tmp/ipykernel_2668818/1038278743.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_st


Fold 1 - Balanced Accuracy: 0.6374, AUROC: 0.7024


Inference Fold 2: 100%|██████████| 52/52 [00:00<00:00, 154.27it/s]



Fold 2 - Balanced Accuracy: 0.6439, AUROC: 0.7108


Inference Fold 3: 100%|██████████| 52/52 [00:00<00:00, 138.96it/s]



Fold 3 - Balanced Accuracy: 0.6529, AUROC: 0.7037


Inference Fold 4: 100%|██████████| 52/52 [00:00<00:00, 159.05it/s]



Fold 4 - Balanced Accuracy: 0.6292, AUROC: 0.6955


Inference Fold 5: 100%|██████████| 52/52 [00:00<00:00, 145.50it/s]


Fold 5 - Balanced Accuracy: 0.6378, AUROC: 0.7077

--- Per-Fold Evaluation Summary (Virchow2) ---
Balanced Accuracy: 0.6403 ± 0.0079
AUROC: 0.7040 ± 0.0052

Saved fold predictions and labels to: virchow2_amibr_test_predictions.pkl


## AtNorM-Br

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from huggingface_hub import login
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from timm.layers import SwiGLUPacked
import pickle

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hugging Face login
login(token="your_huggingface_token_here")  # Replace with your Hugging Face token

# Load Virchow2 model
model_name = "hf-hub:paige-ai/Virchow2"
virchow_model = timm.create_model(model_name, pretrained=True, mlp_layer=SwiGLUPacked, act_layer=torch.nn.SiLU)
virchow_model.eval().to(device)
virchow_config = resolve_data_config(virchow_model.pretrained_cfg, model=virchow_model)
virchow_transform = create_transform(**virchow_config)

# Embedding extractor
def extract_embedding(img_path):
    image = Image.open(img_path).convert("RGB")
    image_tensor = virchow_transform(image).unsqueeze(0).to(device)
    with torch.inference_mode(), torch.autocast(device_type="cuda", dtype=torch.float16):
        output = virchow_model(image_tensor)
        class_token = output[:, 0]
        patch_tokens = output[:, 5:]
        embedding = torch.cat([class_token, patch_tokens.mean(1)], dim=-1).squeeze(0).to(torch.float32)
    return embedding.cpu()

# Inference dataset
class InferenceDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.embeddings = [extract_embedding(p) for p in tqdm(image_paths, desc="Extracting embeddings")]
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Classifier head
class VirchowBinaryClassifier(nn.Module):
    def __init__(self):
        super(VirchowBinaryClassifier, self).__init__()
        self.classifier = nn.Linear(2560, 1)

    def forward(self, x):
        return self.classifier(x)

# Load test data
test_root = "/data/MELBA-AmiBr/Datasets_Stratified/AtNorM-Br"
class_map = {"Atypical": 0, "Normal": 1}
image_paths, labels = [], []

for class_name, label_val in class_map.items():
    class_folder = os.path.join(test_root, class_name)
    for fname in os.listdir(class_folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.tif')):
            image_paths.append(os.path.join(class_folder, fname))
            labels.append(label_val)

# Prepare dataset and dataloader
test_dataset = InferenceDataset(image_paths, labels)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8, pin_memory=True)

# Load trained fold models
num_folds = 5
model_paths = [f"virchow2_linear_probe_fold_{i + 1}_best.pth" for i in range(num_folds)]
models = []

for path in model_paths:
    model = VirchowBinaryClassifier().to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    model.eval()
    models.append(model)

# Inference and evaluation
true_labels = np.array(test_dataset.labels)
fold_bal_accs, fold_aurocs = [], []
fold_probs_dict = {}

for i, model in enumerate(models):
    fold_probs = []

    with torch.no_grad():
        for embeddings, _ in tqdm(test_loader, desc=f"Inference Fold {i + 1}"):
            embeddings = embeddings.to(device)
            logits = model(embeddings)
            probs = torch.sigmoid(logits).squeeze(1).cpu().numpy()
            fold_probs.extend(probs)

    fold_probs = np.array(fold_probs)
    fold_preds = (fold_probs > 0.5).astype(int)

    bal_acc = balanced_accuracy_score(true_labels, fold_preds)
    auroc = roc_auc_score(true_labels, fold_probs)

    fold_bal_accs.append(bal_acc)
    fold_aurocs.append(auroc)

    print(f"\nFold {i + 1} - Balanced Accuracy: {bal_acc:.4f}, AUROC: {auroc:.4f}")

    fold_probs_dict[f"fold_{i + 1}"] = {
        "probs": fold_probs,
        "preds": fold_preds,
        "true_labels": true_labels
    }

# Summary
mean_bal_acc = np.mean(fold_bal_accs)
std_bal_acc = np.std(fold_bal_accs)
mean_auroc = np.mean(fold_aurocs)
std_auroc = np.std(fold_aurocs)

print("\n--- Per-Fold Evaluation Summary (Virchow2) ---")
print(f"Balanced Accuracy: {mean_bal_acc:.4f} ± {std_bal_acc:.4f}")
print(f"AUROC: {mean_auroc:.4f} ± {std_auroc:.4f}")

# Save predictions
output_path = "virchow2_atnorm-br_test_predictions.pkl"
with open(output_path, "wb") as f:
    pickle.dump(fold_probs_dict, f)

print(f"\nSaved fold predictions and labels to: {output_path}")


Extracting embeddings: 100%|██████████| 746/746 [00:23<00:00, 31.47it/s]
/tmp/ipykernel_2668818/2393219343.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_st


Fold 1 - Balanced Accuracy: 0.6828, AUROC: 0.7213


Inference Fold 2: 100%|██████████| 47/47 [00:00<00:00, 76.30it/s]



Fold 2 - Balanced Accuracy: 0.6670, AUROC: 0.7146


Inference Fold 3: 100%|██████████| 47/47 [00:00<00:00, 76.74it/s]



Fold 3 - Balanced Accuracy: 0.6435, AUROC: 0.7118


Inference Fold 4: 100%|██████████| 47/47 [00:00<00:00, 73.69it/s]



Fold 4 - Balanced Accuracy: 0.6758, AUROC: 0.7262


Inference Fold 5: 100%|██████████| 47/47 [00:00<00:00, 77.50it/s]


Fold 5 - Balanced Accuracy: 0.6645, AUROC: 0.7267

--- Per-Fold Evaluation Summary (Virchow2) ---
Balanced Accuracy: 0.6667 ± 0.0133
AUROC: 0.7201 ± 0.0060

Saved fold predictions and labels to: virchow2_atnorm-br_test_predictions.pkl


## AtNorM-MD

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from huggingface_hub import login
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from timm.layers import SwiGLUPacked
import pickle

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hugging Face login
login(token="your_huggingface_token_here")  # Replace with your Hugging Face token

# Load Virchow2 model
model_name = "hf-hub:paige-ai/Virchow2"
virchow_model = timm.create_model(model_name, pretrained=True, mlp_layer=SwiGLUPacked, act_layer=torch.nn.SiLU)
virchow_model.eval().to(device)
virchow_config = resolve_data_config(virchow_model.pretrained_cfg, model=virchow_model)
virchow_transform = create_transform(**virchow_config)

# Embedding extractor
def extract_embedding(img_path):
    image = Image.open(img_path).convert("RGB")
    image_tensor = virchow_transform(image).unsqueeze(0).to(device)
    with torch.inference_mode(), torch.autocast(device_type="cuda", dtype=torch.float16):
        output = virchow_model(image_tensor)
        class_token = output[:, 0]
        patch_tokens = output[:, 5:]
        embedding = torch.cat([class_token, patch_tokens.mean(1)], dim=-1).squeeze(0).to(torch.float32)
    return embedding.cpu()

# Inference dataset
class InferenceDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.embeddings = [extract_embedding(p) for p in tqdm(image_paths, desc="Extracting embeddings")]
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Classifier head
class VirchowBinaryClassifier(nn.Module):
    def __init__(self):
        super(VirchowBinaryClassifier, self).__init__()
        self.classifier = nn.Linear(2560, 1)

    def forward(self, x):
        return self.classifier(x)

# Load test data
test_root = "/data/MELBA-AmiBr/Datasets_Stratified/AtNorM-MD"
class_map = {"Atypical": 0, "Normal": 1}
image_paths, labels = [], []

for class_name, label_val in class_map.items():
    class_folder = os.path.join(test_root, class_name)
    for fname in os.listdir(class_folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.tif')):
            image_paths.append(os.path.join(class_folder, fname))
            labels.append(label_val)

# Prepare dataset and dataloader
test_dataset = InferenceDataset(image_paths, labels)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8, pin_memory=True)

# Load trained fold models
num_folds = 5
model_paths = [f"virchow2_linear_probe_fold_{i + 1}_best.pth" for i in range(num_folds)]
models = []

for path in model_paths:
    model = VirchowBinaryClassifier().to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    model.eval()
    models.append(model)

# Inference and evaluation
true_labels = np.array(test_dataset.labels)
fold_bal_accs, fold_aurocs = [], []
fold_probs_dict = {}

for i, model in enumerate(models):
    fold_probs = []

    with torch.no_grad():
        for embeddings, _ in tqdm(test_loader, desc=f"Inference Fold {i + 1}"):
            embeddings = embeddings.to(device)
            logits = model(embeddings)
            probs = torch.sigmoid(logits).squeeze(1).cpu().numpy()
            fold_probs.extend(probs)

    fold_probs = np.array(fold_probs)
    fold_preds = (fold_probs > 0.5).astype(int)

    bal_acc = balanced_accuracy_score(true_labels, fold_preds)
    auroc = roc_auc_score(true_labels, fold_probs)

    fold_bal_accs.append(bal_acc)
    fold_aurocs.append(auroc)

    print(f"\nFold {i + 1} - Balanced Accuracy: {bal_acc:.4f}, AUROC: {auroc:.4f}")

    fold_probs_dict[f"fold_{i + 1}"] = {
        "probs": fold_probs,
        "preds": fold_preds,
        "true_labels": true_labels
    }

# Summary
mean_bal_acc = np.mean(fold_bal_accs)
std_bal_acc = np.std(fold_bal_accs)
mean_auroc = np.mean(fold_aurocs)
std_auroc = np.std(fold_aurocs)

print("\n--- Per-Fold Evaluation Summary (Virchow2) ---")
print(f"Balanced Accuracy: {mean_bal_acc:.4f} ± {std_bal_acc:.4f}")
print(f"AUROC: {mean_auroc:.4f} ± {std_auroc:.4f}")

# Save predictions
output_path = "virchow2_atnorm-md_test_predictions.pkl"
with open(output_path, "wb") as f:
    pickle.dump(fold_probs_dict, f)

print(f"\nSaved fold predictions and labels to: {output_path}")


Extracting embeddings: 100%|██████████| 2107/2107 [00:48<00:00, 43.40it/s]
/tmp/ipykernel_2668818/1601682304.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_


Fold 1 - Balanced Accuracy: 0.5710, AUROC: 0.6100


Inference Fold 2: 100%|██████████| 132/132 [00:00<00:00, 176.00it/s]



Fold 2 - Balanced Accuracy: 0.5716, AUROC: 0.5856


Inference Fold 3: 100%|██████████| 132/132 [00:00<00:00, 170.25it/s]



Fold 3 - Balanced Accuracy: 0.5581, AUROC: 0.5730


Inference Fold 4: 100%|██████████| 132/132 [00:00<00:00, 174.82it/s]



Fold 4 - Balanced Accuracy: 0.5569, AUROC: 0.5580


Inference Fold 5: 100%|██████████| 132/132 [00:00<00:00, 173.91it/s]


Fold 5 - Balanced Accuracy: 0.5685, AUROC: 0.6126

--- Per-Fold Evaluation Summary (Virchow2) ---
Balanced Accuracy: 0.5652 ± 0.0064
AUROC: 0.5878 ± 0.0211

Saved fold predictions and labels to: virchow2_atnorm-md_test_predictions.pkl
